In [6]:
!pip install pygame


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.3 MB/s  0:00:02a 0:00:010:00:0102


In [4]:
import sys
print(sys.version)

3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]


In [ ]:
# virtual_guitar_app.py
# Requirements: opencv-python, mediapipe, numpy, pygame
# Target webcam resolution: 1280x720
# Optional: place chord images in ./chords/A.png ... etc (used in Tutorial mode)

import cv2
import time
import os
import numpy as np
import pygame

# ------------------ Initialization ------------------
try:
    import mediapipe as mp
except Exception as e:
    raise ImportError("mediapipe required: pip install mediapipe") from e

# init pygame mixer for sound
pygame.mixer.pre_init(44100, -16, 1, 512)
pygame.init()

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Set capture resolution (1280x720)
CAP_INDEX = 0
CAP_W, CAP_H = 1280, 720

# ------------------ Config / Mappings ------------------
NUM_STRINGS = 6
# strings top->bottom indices 0..5
STRING_NAMES = ['E3', 'A3', 'D4', 'G4', 'B4', 'E5']  # descriptive
# frequencies (Hz) for synthetic tones (approx)
STRING_FREQS = {
    0: 164.81,  # E3
    1: 220.00,  # A3
    2: 293.66,  # D4
    3: 392.00,  # G4
    4: 493.88,  # B4
    5: 659.26,  # E5
}

# chord positions (low E -> high e). None = muted, 0=open, >0 fret number
CHORD_POSITIONS = {
    "C": [None, 3, 2, 0, 1, 0],
    "G": [3, 2, 0, 0, 0, 3],
    "D": [None, None, 0, 2, 3, 2],
    "A": [None, 0, 2, 2, 2, 0],
    "E": [0, 2, 2, 1, 0, 0],
    "F": [1, 3, 3, 2, 1, 1],  # barre F
    "B": [2, 2, 4, 4, 4, 2],  # barre B
}

# Tutorial chords list (major only per request)
TUTORIAL_CHORDS = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Song: Twinkle Twinkle (C -> G -> C -> G)
SONG_TWINKLE = [("C", 3.0), ("G", 3.0), ("C", 3.0), ("G", 3.0)]
SONG_STRUM_PATTERN = "↓ ↓ ↑ ↓"  # display-only pattern

# UI / drawing config
STRING_COLORS = [
    (200, 80, 20),  # low E
    (0, 200, 0),
    (20, 20, 200),
    (200, 200, 0),
    (200, 0, 200),
    (0, 200, 200),  # high E
]
BUBBLE_RADIUS = 18
BASE_FRET_X_RATIO = 0.10  # where bubbles start (left)
FRET_SPACING = 36         # pixels per fret
STRUM_AREA_TOP_RATIO = 0.60
STRUM_AREA_BOTTOM_RATIO = 0.68
STRING_COOLDOWN = 0.08    # seconds

# fingertips to consider
FINGERTIP_LANDMARKS = [4, 8, 12, 16, 20]  # thumb, index, middle, ring, pinky

# ------------------ Utility: generate pygame Sound from sine ------------------
def make_sine_sound(freq, duration=0.8, volume=0.5, fs=44100):
    t = np.linspace(0, duration, int(fs * duration), False)
    tone = 0.5 * np.sin(2 * np.pi * freq * t)  # amplitude 0.5
    # ADSR-ish envelope to reduce pops
    attack = int(0.01 * fs)
    decay = int(0.01 * fs)
    sustain_level = 0.9
    sustain = len(t) - (attack + decay)
    env = np.concatenate([
        np.linspace(0, 1, attack),
        np.linspace(1, sustain_level, decay),
        np.ones(max(sustain, 0)) * sustain_level
    ])
    waveform = (tone * env * volume * (2**15 - 1)).astype(np.int16)
    sound = pygame.sndarray.make_sound(waveform)
    return sound

# preload sounds
preloaded_sounds = {}
for idx, f in STRING_FREQS.items():
    preloaded_sounds[idx] = make_sine_sound(f, duration=1.0, volume=0.6)

# ------------------ UI Button helper ------------------
class Button:
    def __init__(self, rect, label, color=(40,40,40), hover_color=(80,80,80), text_color=(255,255,255)):
        self.rect = rect  # (x,y,w,h)
        self.label = label
        self.color = color
        self.hover_color = hover_color
        self.text_color = text_color
        self.hover = False
    def draw(self, img):
        x,y,w,h = self.rect
        col = self.hover_color if self.hover else self.color
        cv2.rectangle(img, (x,y), (x+w, y+h), col, -1)
        cv2.rectangle(img, (x,y), (x+w, y+h), (200,200,200), 2)
        # center text
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.9
        thickness = 2
        (tw, th), _ = cv2.getTextSize(self.label, font, scale, thickness)
        tx = x + (w - tw)//2
        ty = y + (h + th)//2
        cv2.putText(img, self.label, (tx, ty), font, scale, self.text_color, thickness, cv2.LINE_AA)
    def is_inside(self, px, py):
        x,y,w,h = self.rect
        return x <= px <= x+w and y <= py <= y+h

# ------------------ App State ------------------
STATE_HOME = "HOME"
STATE_SONG_SELECT = "SONG_SELECT"
STATE_SONG_PLAY = "SONG_PLAY"
STATE_TUTORIAL_SELECT = "TUTORIAL_SELECT"
STATE_TUTORIAL = "TUTORIAL"

state = STATE_HOME
selected_song = None
selected_chord = 'C'  # default
# Song progression state
song_list = {"Twinkle": SONG_TWINKLE}
current_song = None
song_index = 0
song_start = None

# Strum / hold tracking
last_play_times = [0.0] * NUM_STRINGS
# held_by[string] = set of finger keys pressing bubble
held_by = {i:set() for i in range(NUM_STRINGS)}
# fingertip previous Y to detect crossing strum band
fingertip_prev_y = {}

# mediapipe hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.55, min_tracking_confidence=0.55)

# clickable UI buttons (will be created after knowing capture size)
buttons = []

# mouse callback variables
mouse_x, mouse_y, mouse_clicked = 0,0,False

def mouse_cb(event, x, y, flags, param):
    global mouse_x, mouse_y, mouse_clicked
    mouse_x, mouse_y = x, y
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_clicked = True

# ------------------ Drawing helpers ------------------
def compute_string_positions(H):
    top = int(H * 0.20)
    bottom = int(H * 0.80)
    return [ int(top + (i/(NUM_STRINGS-1))*(bottom-top)) for i in range(NUM_STRINGS) ]

def draw_strings_and_bubbles(frame, string_positions, chord_name, held_by_local):
    H, W = frame.shape[:2]
    overlay = frame.copy()
    # draw strings
    for i, y in enumerate(string_positions):
        cv2.line(overlay, (int(W*0.05), y), (int(W*0.95), y), STRING_COLORS[i], 4)
    # draw strum band
    area_top = int(H * STRUM_AREA_TOP_RATIO)
    area_bottom = int(H * STRUM_AREA_BOTTOM_RATIO)
    cv2.rectangle(overlay, (0, area_top), (W, area_bottom), (60,60,60), 1)
    cv2.putText(overlay, "STRUM AREA", (W - 220, area_top + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (180,180,180), 1)

    # draw chord bubbles at left side
    base_x = int(W * BASE_FRET_X_RATIO)
    if chord_name in CHORD_POSITIONS:
        shape = CHORD_POSITIONS[chord_name]
        for i, fret in enumerate(shape):
            sy = string_positions[i]
            if fret is None:
                # muted: show 'x' slightly left
                cv2.putText(overlay, "x", (base_x - 20, sy + 6), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            elif fret == 0:
                cx = base_x
                cv2.circle(overlay, (cx, sy), BUBBLE_RADIUS, (255,255,255), 2)  # open
            else:
                cx = base_x + int(fret * FRET_SPACING)
                # filled if held
                if len(held_by_local[i]) > 0:
                    cv2.circle(overlay, (cx, sy), BUBBLE_RADIUS, (0,200,0), -1)
                else:
                    cv2.circle(overlay, (cx, sy), BUBBLE_RADIUS, (0,200,0), 2)
                cv2.putText(overlay, str(fret), (cx-6, sy+6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0) if len(held_by_local[i])>0 else (0,200,0), 1)

    # blend
    cv2.addWeighted(overlay, 0.78, frame, 0.22, 0, frame)

def draw_top_text(frame, text):
    H, W = frame.shape[:2]
    cv2.putText(frame, text, (W//2 - 200, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (230,230,20), 2, cv2.LINE_AA)

def draw_strum_pattern(frame, pattern_text):
    H, W = frame.shape[:2]
    cv2.putText(frame, pattern_text, (W - 420, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (180,200,255), 2)

# ------------------ Detection helpers ------------------
def fingertip_key(hand_idx, lm_idx):
    return (hand_idx, lm_idx)

def update_held_by_from_fingertips(fingertip_positions, string_positions, chord_name):
    """fingertip_positions: dict key->(x,y). returns updated held_by mapping"""
    # reset held_by
    for i in range(NUM_STRINGS):
        held_by[i].clear()
    if chord_name not in CHORD_POSITIONS:
        return
    base_x = int(CAP_W * BASE_FRET_X_RATIO)
    shape = CHORD_POSITIONS[chord_name]
    for key, (x_px, y_px) in fingertip_positions.items():
        for s_idx, fret in enumerate(shape):
            # bubble center
            if fret is None:
                cx = base_x
                cy = string_positions[s_idx]
            elif fret == 0:
                cx = base_x
                cy = string_positions[s_idx]
            else:
                cx = base_x + int(fret * FRET_SPACING)
                cy = string_positions[s_idx]
            # distance test
            if (x_px - cx)**2 + (y_px - cy)**2 <= (BUBBLE_RADIUS*1.1)**2:
                held_by[s_idx].add(key)
                # a fingertip can only press one bubble at a time; break
                break

def detect_and_play_strum(fingertip_positions, string_positions):
    """Check fingertip sweep across strum band and play held strings crossed."""
    area_top = int(CAP_H * STRUM_AREA_TOP_RATIO)
    area_bottom = int(CAP_H * STRUM_AREA_BOTTOM_RATIO)
    for key, (x_px, y_px) in fingertip_positions.items():
        prev_y = fingertip_prev_y.get(key, None)
        if prev_y is None:
            fingertip_prev_y[key] = y_px
            continue
        crossed = False
        direction = None
        if prev_y < area_top and y_px >= area_top:
            crossed = True; direction = "down"
        elif prev_y < area_bottom and y_px >= area_bottom:
            crossed = True; direction = "down"
        elif prev_y > area_bottom and y_px <= area_bottom:
            crossed = True; direction = "up"
        elif prev_y > area_top and y_px <= area_top:
            crossed = True; direction = "up"
        if crossed:
            low = min(prev_y, y_px)
            high = max(prev_y, y_px)
            strings_crossed = []
            for s_idx, s_y in enumerate(string_positions):
                if low - 1 <= s_y <= high + 1:
                    strings_crossed.append((s_idx, s_y))
            # play in crossing order depending on direction
            strings_crossed.sort(key=lambda t: t[1], reverse=(direction=="up"))
            for s_idx, _ in strings_crossed:
                if len(held_by[s_idx]) > 0:
                    now = time.time()
                    if now - last_play_times[s_idx] >= STRING_COOLDOWN:
                        last_play_times[s_idx] = now
                        # play preloaded sound
                        snd = preloaded_sounds.get(s_idx)
                        if snd: snd.play()
        fingertip_prev_y[key] = y_px

# ------------------ UI / Mode functions ------------------
def create_home_buttons(W, H):
    # center two buttons horizontally
    btn_w = 360; btn_h = 110
    gap = 80
    x0 = (W - (btn_w*2 + gap))//2
    y0 = int(H*0.30)
    b1 = Button((x0, y0, btn_w, btn_h), "Song Mode", color=(30,30,30), hover_color=(70,70,70))
    b2 = Button((x0 + btn_w + gap, y0, btn_w, btn_h), "Tutorial Mode", color=(30,30,30), hover_color=(70,70,70))
    return [b1, b2]

def create_song_select_buttons(W,H):
    btn_w = 420; btn_h = 80
    x = (W - btn_w)//2
    y = int(H*0.30)
    b_song = Button((x, y, btn_w, btn_h), "Twinkle Twinkle", color=(40,40,40), hover_color=(80,80,80))
    b_back = Button((20, 20, 140, 50), "Back", color=(30,30,30))
    return [b_song, b_back]

def create_tutorial_select_buttons(W,H):
    # grid of chord buttons
    btns = []
    btn_w = 200; btn_h = 80; gapx = 32; gapy = 20
    cols = 4
    start_x = 120; start_y = 160
    for idx, chord in enumerate(TUTORIAL_CHORDS):
        r = idx // cols; c = idx % cols
        x = start_x + c*(btn_w + gapx)
        y = start_y + r*(btn_h + gapy)
        btns.append(Button((x,y,btn_w,btn_h), chord, color=(30,30,30), hover_color=(70,70,70)))
    btns.append(Button((20,20,120,50), "Back", color=(30,30,30)))
    return btns

# ------------------ Main loop ------------------
def main():
    global state, buttons, selected_song, current_song, song_index, song_start
    global selected_chord, mouse_clicked, mouse_x, mouse_y, song_list, song_mode
    # open capture
    cap = cv2.VideoCapture(CAP_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAP_W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAP_H)
    actual_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    actual_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("Camera resolution:", actual_w, "x", actual_h)

    cv2.namedWindow("Virtual Guitar")
    cv2.setMouseCallback("Virtual Guitar", mouse_cb)

    # initial buttons for home
    buttons = create_home_buttons(actual_w, actual_h)

    # state-specific button collections
    song_select_buttons = create_song_select_buttons(actual_w, actual_h)
    tutorial_select_buttons = create_tutorial_select_buttons(actual_w, actual_h)

    # for storing latest fingertip positions in frame
    fingertip_positions = {}  # key -> (x,y)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("camera read failed")
            break
        frame = cv2.flip(frame, 1)
        H, W = frame.shape[:2]

        # handle mouse hover states
        for b in buttons:
            b.hover = b.is_inside(mouse_x, mouse_y)

        # Draw UI per state
        display_frame = frame.copy()

        if state == STATE_HOME:
            # draw title and buttons
            draw_top_text(display_frame, "Virtual Guitar — Home")
            for b in buttons:
                b.draw(display_frame)
            # click handling
            if mouse_clicked:
                for b in buttons:
                    if b.is_inside(mouse_x, mouse_y):
                        if b.label == "Song Mode":
                            state = STATE_SONG_SELECT
                            buttons = song_select_buttons
                        elif b.label == "Tutorial Mode":
                            state = STATE_TUTORIAL_SELECT
                            buttons = tutorial_select_buttons
                        mouse_clicked = False
                        break

        elif state == STATE_SONG_SELECT:
            draw_top_text(display_frame, "Song Mode — Choose a song")
            for b in buttons:
                b.draw(display_frame)
            if mouse_clicked:
                mouse_clicked = False
                for b in buttons:
                    if b.is_inside(mouse_x, mouse_y):
                        if b.label == "Twinkle Twinkle":
                            selected_song = "Twinkle"
                            current_song = song_list[selected_song]
                            song_index = 0
                            song_start = time.time()
                            # set first chord
                            selected_chord = current_song[0][0]
                            state = STATE_SONG_PLAY
                            # reset play trackers
                            for k in fingertip_prev_y.keys(): fingertip_prev_y[k]=None
                            break
                        elif b.label == "Back":
                            state = STATE_HOME
                            buttons = create_home_buttons(W,H)
                            break

        elif state == STATE_SONG_PLAY:
            # show strings and bubbles for current chord
            string_positions = compute_string_positions(H)
            draw_strings_and_bubbles(display_frame, string_positions, selected_chord, held_by)
            # top text: song & chord
            draw_top_text(display_frame, f"Song: {selected_song}   Now: {selected_chord}")
            draw_strum_pattern(display_frame, SONG_STRUM_PATTERN)

            # process hands
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)
            fingertip_positions.clear()
            if results.multi_hand_landmarks and results.multi_handedness:
                for hand_idx, (landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                    mp_drawing.draw_landmarks(display_frame, landmarks, mp_hands.HAND_CONNECTIONS)
                    for lm_idx in FINGERTIP_LANDMARKS:
                        lm = landmarks.landmark[lm_idx]
                        x_px = int(lm.x * W)
                        y_px = int(lm.y * H)
                        fingertip_positions[fingertip_key(hand_idx, lm_idx)] = (x_px, y_px)
                        # draw fingertip
                        cv2.circle(display_frame, (x_px,y_px), 5, (0,255,0), -1)

            # compute holds and strums
            update_held_by_from_fingertips(fingertip_positions, string_positions, selected_chord)
            detect_and_play_strum(fingertip_positions, string_positions)

            # check if current chord has been successfully played (i.e., at least one held string got sounded)
            # we detect success by whether any string belonging to the chord was played in the recent short window
            # track using last_play_times timestamps
            chord_shape = CHORD_POSITIONS.get(selected_chord, [])
            # list strings that should be held (non-None and > =0)
            should_hold_strings = [i for i, v in enumerate(chord_shape) if v is not None]
            # success if any of those were played in last 1 second AND they were held at time of play
            success = False
            now = time.time()
            for s in should_hold_strings:
                if now - last_play_times[s] < 1.0 and len(held_by[s])>0:
                    success = True
                    break
            if success:
                # advance chord
                song_index += 1
                if song_index >= len(current_song):
                    # song ended: go back to song select
                    state = STATE_SONG_SELECT
                    buttons = song_select_buttons
                else:
                    selected_chord = current_song[song_index][0]
                    song_start = time.time()
                    # clear prev y so next strum detection fresh
                    fingertip_prev_y.clear()

            # draw back button
            back_btn = Button((20,20,120,50), "Back")
            back_btn.draw(display_frame)
            if mouse_clicked:
                mouse_clicked = False
                if back_btn.is_inside(mouse_x, mouse_y):
                    state = STATE_SONG_SELECT
                    buttons = song_select_buttons

        elif state == STATE_TUTORIAL_SELECT:
            draw_top_text(display_frame, "Tutorial Mode — Choose a chord")
            for b in buttons:
                b.draw(display_frame)
            if mouse_clicked:
                mouse_clicked = False
                for b in buttons:
                    if b.is_inside(mouse_x, mouse_y):
                        if b.label == "Back":
                            state = STATE_HOME
                            buttons = create_home_buttons(W,H)
                            break
                        else:
                            selected_chord = b.label
                            state = STATE_TUTORIAL
                            # reset trackers
                            fingertip_prev_y.clear()
                            for i in range(NUM_STRINGS): last_play_times[i]=0.0
                            break

        elif state == STATE_TUTORIAL:
            string_positions = compute_string_positions(H)
            draw_strings_and_bubbles(display_frame, string_positions, selected_chord, held_by)
            draw_top_text(display_frame, f"Tutorial — {selected_chord} Major")
            draw_strum_pattern(display_frame, "Pattern: ↓ ↓ ↑ ↓")

            # show chord image top-left if available
            instr_w = int(W*0.18); instr_h = int(H*0.22)
            img_path = os.path.join("chords", f"{selected_chord}.png")
            if os.path.exists(img_path):
                img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
                if img is not None:
                    ih, iw = img.shape[:2]
                    scale = min(instr_w/iw, instr_h/ih)
                    nw, nh = int(iw*scale), int(ih*scale)
                    r = cv2.resize(img, (nw, nh))
                    display_frame[6:6+nh, 6:6+nw] = r

            # process hands
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)
            fingertip_positions.clear()
            if results.multi_hand_landmarks and results.multi_handedness:
                for hand_idx, (landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                    mp_drawing.draw_landmarks(display_frame, landmarks, mp_hands.HAND_CONNECTIONS)
                    for lm_idx in FINGERTIP_LANDMARKS:
                        lm = landmarks.landmark[lm_idx]
                        x_px = int(lm.x * W)
                        y_px = int(lm.y * H)
                        fingertip_positions[fingertip_key(hand_idx, lm_idx)] = (x_px, y_px)
                        cv2.circle(display_frame, (x_px,y_px), 5, (0,255,0), -1)

            update_held_by_from_fingertips(fingertip_positions, string_positions, selected_chord)
            detect_and_play_strum(fingertip_positions, string_positions)

            # Draw Back button
            back_btn = Button((20,20,120,50), "Back")
            back_btn.draw(display_frame)
            if mouse_clicked:
                mouse_clicked = False
                if back_btn.is_inside(mouse_x, mouse_y):
                    state = STATE_TUTORIAL_SELECT
                    buttons = tutorial_select_buttons

        # reset mouse click after handling frame
        # show frame
        cv2.imshow("Virtual Guitar", display_frame)
        # consume click event once per loop
        if mouse_clicked:
            # small guard: keep it until processed above; if not used, reset
            # will be reset after each relevant branch
            pass

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

if __name__ == "__main__":
    main()


/home/aditinair/anaconda3/envs/guitar/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


I0000 00:00:1762014851.497324   41048 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1762014851.505903   41098 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Vega 3 Graphics (radeonsi, raven2, ACO, DRM 3.61, 6.14.0-29-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1762014851.528087   41086 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762014851.555754   41087 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/aditinair/anaconda3/envs/guitar/lib/python3.11/site-packages/cv2/qt/plugins"


Camera resolution: 640 x 480


/home/aditinair/anaconda3/envs/guitar/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [1]:
import cv2

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Could not open webcam.")
else:
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Webcam resolution: {width} x {height}")
cap.release()


Webcam resolution: 640 x 480
